# Linear Regression
Linear regression is a statistical method that models a relationship between s dependent variables and one or more independant variables

**One independant variable**

`y = mx + c` where `y` can be house prices and `x` can be house size

**Multiple independant variables**

`y = w1x1 + w2x2 .... + w6x6 + b` where `y` is the house prices, `x1, x2..` are features like house size, number of bedrooms and distance to city center, and `w1,w2...` are weights which determine the influence of each feature and `b` is the bias, which adjusts the baseline prediction (shifts up or down without changing slope)

## Cost Function (MSE - cost function)
Minimise MSE to get the best weight and bias

$$
MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
$$

Where,

$ N $ is the number of data points,

$ y_i $ is the actual value we are trying to predict,i.e, the target variable,

$ hat{y}_i $ is the predicted value from the regression equation

MSE measures how far the predictions are from actual values. A smaller MSE means a better fit.

### Side Note
New terms I learnt were loss function, cost function, and objective function
* **Loss function** is defined on data point, prediction and label and measures the penalty (Squared loss, hinge loss, 0/1 loss)

* **Cost function** is a bit more general, it is a sum of loss functions over your training set plus some model complexity penalty (regularization) (MSE, SVM cost function)

* **Objective function** is the most general term for any function that is optimized during training


## Gradient Descent Optimation (MSE reductions)
